# training

> Training loop

In [ ]:
#|default_exp training

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from isaacai.utils import *
from isaacai.dataloaders import *
from isaacai.models import *

from matplotlib import pyplot as plt
from fastcore.all import *
import torch
from torch import nn
from torch import Tensor
from datasets import load_dataset
from torch.utils.data import DataLoader
import pandas as pd 
import numpy as np
from datasets import Dataset
from torcheval.metrics import MulticlassAccuracy,Mean

In [ ]:
dls = DataLoaders(*load_fashion_mnist(sample=200))
dls.x_name,dls.y_name = 'image','label'

Found cached dataset fashion_mnist (/home/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/8d6c32399aa01613d96e2cbc9b13638f359ef62bb33612b077b4c247f6ef99c1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model = SimpleNet(784,32,10)
loss_func = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
training_stats = pd.DataFrame()

In [ ]:
dls.x_name,dls.y_name = 'image','label'

In [ ]:
#| export
def run_callbacks(callbacks, method_name, trainer=None):
    for callback in sorted(callbacks, key=lambda x: getattr(x, 'order',0)):
        callback_method = getattr(callback, method_name,None)
        if callback_method is not None: callback_method(trainer)

In [ ]:
#| export
class ProgressCB:
    def __init__(self, **metrics):
        self.metrics = metrics
        self.loss_train, self.loss_valid = Mean(), Mean()
        self.stats_epoch = L()
        
    def after_batch(self,trainer):
        # Collect loss, metrics and store
        if trainer.training: self.loss_train.update(trainer.loss)
        else: 
            self.loss_valid.update(trainer.loss)
            for name, metric in self.metrics.items():
                self.metrics[name].update(trainer.preds,trainer.batch[trainer.dls.y_name])
    
    def after_epoch(self,trainer):
        # compute metrics and append to epoch stats and display
        
        _stats = {'train_loss':float(self.loss_train.compute()),
                  'valid_loss':float(self.loss_valid.compute())}
        _stats.update({name:float(metric.compute()) for name, metric in self.metrics.items()})
     
        
        self.stats_epoch.append(_stats)
                 
        self.loss_train.reset(); self.loss_valid.reset(); [metric.reset() for _,metric in self.metrics.items()];
        
        print(trainer.epoch,_stats)

        

In [ ]:
#| export 
class Trainer:
    def __init__(self, dls, loss_func, opt_func, model, callbacks):
        store_attr()

    def one_batch(self):
        self.run_callbacks('before_batch')
        self.preds = self.model(self.batch[self.dls.x_name])
        self.loss = self.loss_func(self.preds, self.batch[self.dls.y_name])
        if self.training:
            self.loss.backward()
            self.opt_func.step()
            self.opt_func.zero_grad()
        self.run_callbacks('after_batch')

    def one_epoch(self):
        self.run_callbacks('before_epoch')
        
        self.model.train()
        self.run_callbacks('before_train')
        for self.batch in self.dls.train: self.one_batch()
        self.run_callbacks('after_train')

        self.model.eval()
        self.run_callbacks('before_valid')
        for self.batch in self.dls.valid: self.one_batch()
        self.run_callbacks('after_valid')
        
        self.run_callbacks('after_epoch')

    def fit(self, epochs=3):
        self.run_callbacks('before_fit')
        for self.epoch in range(epochs): self.one_epoch()
        self.run_callbacks('after_fit')

    @property
    def training(self): return self.model.training

    def run_callbacks(self,method_name): run_callbacks(self.callbacks,method_name,self)

In [ ]:
trainer = Trainer(dls,loss_func, opt, model, callbacks=[ProgressCB(Accuracy=MulticlassAccuracy())])

In [ ]:
trainer.fit()

0 {'train_loss': 0.6038940727432384, 'valid_loss': 0.45580716927187515, 'Accuracy': 0.8454009890556335}
1 {'train_loss': 0.45173856063812395, 'valid_loss': 0.48607442107061494, 'Accuracy': 0.8327942490577698}
2 {'train_loss': 0.4198705881953319, 'valid_loss': 0.4158861208262431, 'Accuracy': 0.8545243144035339}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()